# Start

In [300]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [301]:
import pandas as pd
import plotly.express as px
import regex as re
import time
import random

from tqdm.notebook import tqdm

In [302]:
test_links = [
    'https://d-nb.info/gnd/118505602', # Bachmann
    'https://d-nb.info/gnd/118516906', # Büchner
    'https://d-nb.info/gnd/118519859', # Celan
    'https://d-nb.info/gnd/118523392', # Dahn
    'https://d-nb.info/gnd/118527908', # Dürrenmatt
    'https://d-nb.info/gnd/118536109', # Frisch
    'https://d-nb.info/gnd/118585916', # Musil
]

In [303]:
gnd_authors_to_standard_authors = {
    "Aesopus": "Äsop",
    'Apuleius, Madaurensis': 'Apuleius',
    'Ava, Frau' : 'Frau Ava',
    'Dietmar, von Aist': 'Dietmar von Aist',
    'Eckhart, Meister': 'Meister Eckhart',
    'Elisabeth, Nassau-Saarbrücken, Gräfin': 'Elisabeth von Lothringen',
    'Erasmus, Desiderius': 'Erasmus von Rotterdam',
    'Feuerbach, Paul Johann Anselm, Ritter von' : 'Feuerbach, Anselm von',
    'Fouqué, Friedrich de La Motte-' : 'Fouqué, Friedrich de la Motte',
    'Gottfried, von Straßburg' : 'Gottfried von Straßburg',
    'Hahn-Hahn, Ida, Gräfin': 'Ida Hahn-Hahn',
    'Hartmann, von Aue' : 'Hartmann von Aue',
    'Heinrich, der Gleißner' : 'Heinrich der Gleißner',
    'Heinrich, von Meißen' : 'Frauenlob',
    'Heinrich, von Morungen' : 'Heinrich von Morungen',
    'Heinrich, von Veldeke' : 'Heinrich von Veldeke',
    "Homerus" : "Homer",
    'Knigge, Adolph, Freiherr': 'Knigge, Adolph',
    'Konrad, der Pfaffe': 'Pfaffe Konrad',
    'Konrad, von Würzburg': 'Konrad von Würzburg',
    'Lamprecht, der Pfaffe' : 'Pfaffe Lamprecht',
    "Marlitt, E." : "Marlitt, Eugenie",
    "Mechthild, von Magdeburg": "Mechthild von Magdeburg",
    "Otfrid, von Weißenburg": "Otfrid von Weißenburg",
    'Platen, August, Graf von' : 'Platen, August von',
    "Reinbot, von Durne": "Reinbot von Durne",
    "Reinmar, der Alte": "Reinmar der Alte",
    "Seneca, Lucius Annaeus, Philosophus": "Seneca",
    "Schlegel, Dorothea von": "Schlegel, Dorothea",
    "Sophocles": "Sophokles",
    'Thüring, von Ringoltingen' : 'Thüring von Ringoltingen',
    'Ulrich, von Lichtenstein' : 'Ulrich von Liechtenstein',
    "Vergilius Maro, Publius": "Vergil",
    'Walther, von der Vogelweide' : 'Walther von der Vogelweide',
    "Werner, der Gärtner": "Werner der Gärtner",
    "Wolfram, von Eschenbach": "Wolfram von Eschenbach",
}

In [304]:
data_authors = pd.DataFrame()
data_authors.index.name = 'GND'

# Leselisten

### create leselisten_authors from raw

In [ ]:
mode = 'gewichtet'

In [ ]:
leselisten_titles = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/raw/leselisten_raw.csv", sep = ";")

In [ ]:
# Filter
leselisten_titles = leselisten_titles.query("GND.notna()")
leselisten_titles = leselisten_titles[~leselisten_titles['Autor'].str.startswith('$', na = False)]

In [ ]:
leselisten_titles.head()

,Autor,GND,Geschlecht,Titel,Jahreszahl,Jahreszahl_Statistik,Gattung,Modul_Zeit_vor_17.Jh.,Modul_Sprache_(international),Modul_KJL,...,Salzburg,Stuttgart,Stuttgart 2022,Trier,Tübingen,Wien,Wuppertal,Würzburg,Würzburg_2019,Zürich
28,"Hage, Volker (Hg.)",https://d-nb.info/gnd/115807454,NaN,Lyrik für Leser. Deutsche Gedichte der siebzig...,1981,1981.0,Lyrik,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,"Marsch, Edgar (Hg.)",https://d-nb.info/gnd/13345052X,NaN,Moderne deutsche Naturlyrik,1980,1980.0,Lyrik,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,(Pfaffe Konrad),https://d-nb.info/gnd/118565060,m,Rolandslied,12./13. Jh.,1172.0,NaN,MA,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,"Abonji, Melinda Nadj",https://d-nb.info/gnd/129396532,w,Tauben fliegen auf,2010,2010.0,Prosa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,"Achleitner, Friedrich",https://d-nb.info/gnd/119369125,m,Quadratroman,1973,1973.0,Prosa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
leselisten_titles['Autor'] = [x.rstrip() if pd.notna(x) else x for x in leselisten_titles['Autor']]
leselisten_titles['Jahreszahl_Statistik'] = leselisten_titles['Jahreszahl_Statistik'].astype(float)

In [ ]:
leselisten_titles = leselisten_titles.rename(columns = {
    'Modul_Zeit_vor_17.Jh.' : 'Modul_Zeit',
    'Modul_Sprache_(international)' : 'Modul_Sprache',
    'Modul: KJL' : 'Modul_KJL',
})

In [ ]:
leselisten_dict = {
    'Aachen' : 'Aachen1',
    'Berlin' : 'FU Berlin',
    'Innsbruck' : 'Innsbruck1',
    'Innsbruck 2023' : 'Innsbruck2',
    'Köln Fundamentum' : 'Köln',
    'LA Aachen' : 'Aachen2',
    'Stuttgart' : 'Stuttgart1',
    'Stuttgart 2022' : 'Stuttgart2',
    'Würzburg' : 'Würzburg1',
    'Würzburg_2019' : 'Würzburg2',
}

leselisten_titles = leselisten_titles.rename(columns=leselisten_dict)

In [ ]:
exceptions = [
    'Autor', 'GND', 'Geschlecht', 'Titel', 'Jahreszahl', 'Jahreszahl_Statistik', 'Gattung',
    'Modul_Zeit', 'Modul_Sprache', 'Modul_KJL', 'Dekade', 'Jahrhundert', 'sum'
]
leselisten = [x for x in leselisten_titles.columns.tolist() if x not in exceptions]
leselisten_prefix = ['Leselisten_'+x for x in leselisten]

In [ ]:
def convert_to_float(frac_str):
    if type(frac_str) == str:
        frac_str = re.sub(",", ".", frac_str)
    try:
        return float(frac_str)
    except ValueError:
        frac_str = re.sub("⁄", "/", frac_str)
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

leselisten_titles[leselisten] = leselisten_titles[leselisten].applymap(convert_to_float)
leselisten_titles[leselisten] = leselisten_titles[leselisten].astype(float)

In [ ]:
# alles über 0 auf 1 setzen, falls ungewichtet
if mode == 'ungewichtet':
    leselisten_titles[leselisten] = leselisten_titles[leselisten].notnull().astype('int')

# NaN mit 0 ersetzen
leselisten_titles[leselisten] = leselisten_titles[leselisten].fillna(0)

In [ ]:
# mit mehreren Autor:innen pro Text umgehen
for multi_author_index in leselisten_titles.query("GND.str.contains('\+')").index:
  multi_author_series = leselisten_titles.loc[multi_author_index].copy()
  leselisten_titles = leselisten_titles.drop(multi_author_index)

  multi_author_authors = [x.strip() for x in multi_author_series['Autor'].split("/")]
  multi_author_GNDs = [x.strip() for x in multi_author_series['GND'].split("+")]

  for single_author, single_GND in zip(multi_author_authors, multi_author_GNDs):
    single_author_series = multi_author_series.copy()
    single_author_series['Autor'] = single_author
    single_author_series['GND'] = single_GND
    # single_author_series[leselisten] = single_author_series[leselisten]/len(multi_author_authors)
    leselisten_titles = pd.concat([leselisten_titles, pd.DataFrame(single_author_series).T])

In [ ]:
leselisten_authors = pd.DataFrame()
leselisten_authors.index = leselisten_titles.drop_duplicates(subset='GND')['GND']
leselisten_authors['Leselisten_Autor'] = leselisten_titles.drop_duplicates(subset='GND')['Autor'].tolist()
leselisten_authors[leselisten_prefix] = leselisten_titles.groupby('GND')[leselisten].sum()

In [ ]:
# Jeden Wert durch Gesamtzahl aller Titel in jeweiliger Leseiste teilen
# leselisten_authors.loc[:, leselisten_prefix] = leselisten_authors.loc[:, leselisten_prefix].apply(lambda x: x / x.sum())

In [ ]:
leselisten_authors['Leselisten_Summe'] = leselisten_authors[leselisten_prefix].sum(axis=1)

In [ ]:
leselisten_authors = leselisten_authors.sort_values(by = 'Leselisten_Autor')

In [ ]:
this_test_links = [x for x in test_links if x in leselisten_authors.index]
leselisten_authors.loc[this_test_links][[
    'Leselisten_Autor',
    'Leselisten_Göttingen', 'Leselisten_Würzburg1',
    'Leselisten_Summe'
]]

,Leselisten_Autor,Leselisten_Göttingen,Leselisten_Würzburg1,Leselisten_Summe
GND,,,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",1.0,2.0,78.389635
https://d-nb.info/gnd/118516906,"Büchner, Georg",2.0,3.0,94.503444
https://d-nb.info/gnd/118519859,"Celan, Paul",0.0,3.0,59.005877
https://d-nb.info/gnd/118523392,"Dahn, Felix",0.0,0.0,1.0
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich",1.0,1.0,51.812558
https://d-nb.info/gnd/118536109,"Frisch, Max",1.0,2.0,63.029624
https://d-nb.info/gnd/118585916,"Musil, Robert",1.0,4.0,61.488971


In [ ]:
leselisten_authors[leselisten_prefix].sum().head()

Leselisten_Aachen2      24.773429
Leselisten_Aachen1       61.42512
Leselisten_Augsburg         109.0
Leselisten_FU Berlin        260.0
Leselisten_Bochum            98.0
dtype: object

In [ ]:
leselisten_authors.shape

(911, 44)

In [ ]:
leselisten_authors.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/01_leselisten.csv")

### Import and Merge

In [305]:
leselisten_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/01_leselisten.csv", index_col = [0])

In [306]:
cols_to_join = leselisten_authors.columns.difference(data_authors.columns)
data_authors = data_authors.join(leselisten_authors[cols_to_join], how = 'outer')

In [307]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'Leselisten_Autor',
    'Leselisten_Göttingen', 'Leselisten_Würzburg1',
    'Leselisten_Summe',
]]

,Leselisten_Autor,Leselisten_Göttingen,Leselisten_Würzburg1,Leselisten_Summe
GND,,,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",1.0,2.0,78.389635
https://d-nb.info/gnd/118516906,"Büchner, Georg",2.0,3.0,94.503444
https://d-nb.info/gnd/118519859,"Celan, Paul",0.0,3.0,59.005877
https://d-nb.info/gnd/118523392,"Dahn, Felix",0.0,0.0,1.000000
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich",1.0,1.0,51.812558
https://d-nb.info/gnd/118536109,"Frisch, Max",1.0,2.0,63.029624
https://d-nb.info/gnd/118585916,"Musil, Robert",1.0,4.0,61.488971


# Schule

### create schule_authors from raw

In [ ]:
schule_titles = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/raw/schule_raw.csv", sep=";")

In [ ]:
schule_titles = schule_titles.query("GND.notna()")

In [ ]:
schule_titles.head()

,ID,Bundesland,Nachname_original,Nachname,GND,Geschlecht,Titel,Erscheinungsjahr,Erscheinungsjahr_k,Lit.preis_Name,Lit.preis_Autor_in_Name,Jug.literatur
0,2508,Baden-Württemberg,Äsop,Äsop,https://d-nb.info/gnd/118647180,männlich,Fabeln,unbekannt,unbekannt,NaN,NaN,nein
1,2447,Sachsen-Anhalt,Äsop,Äsop,https://d-nb.info/gnd/118647180,männlich,Fabeln,unbekannt,unbekannt,NaN,NaN,nein
2,1778,Baden-Württemberg,Achebe,Achebe,https://d-nb.info/gnd/118646680,männlich,Okonkwo oder Das Alte stürzt,1959,1958,NaN,NaN,nein
3,297,Hessen,Ade,Ade,https://d-nb.info/gnd/138053669,weiblich,Toni Erdmann (Film),2016,2017,"(mehrere Filmpreise, siehe Wikipedia)",NaN,ja
4,3,Hamburg,Ade,Ade,https://d-nb.info/gnd/138053669,weiblich,Toni Erdmann (Film),2016,2016,"(mehrere Filmpreise, siehe Wikipedia)",NaN,ja


In [ ]:
bundeslaender = schule_titles['Bundesland'].unique()
bundeslaender_prefix = ['Schule_'+x for x in bundeslaender]

In [ ]:
# mit mehreren Autor:innen pro Text umgehen
for multi_author_index in schule_titles.query("GND.str.contains('\+')").index:
  multi_author_series = schule_titles.loc[multi_author_index].copy()
  schule_titles = schule_titles.drop(multi_author_index)

  multi_author_authors = [x.strip() for x in multi_author_series['Nachname'].split("/")]
  multi_author_GNDs = [x.strip() for x in multi_author_series['GND'].split("+")]

  for single_author, single_GND in zip(multi_author_authors, multi_author_GNDs):
    single_author_series = multi_author_series.copy()
    single_author_series['Nachname'] = single_author
    single_author_series['GND'] = single_GND
    schule_titles = pd.concat([schule_titles, pd.DataFrame(single_author_series).T])

In [ ]:
schule_authors = pd.DataFrame()

for gnd_link in tqdm(schule_titles['GND'].unique()):
  meta_author = schule_titles.query("GND == @gnd_link")
  schule_authors.loc[gnd_link, 'Schule_Autor'] = meta_author['Nachname'].tolist()[0]
  for bundesland in bundeslaender:
    meta_author_bundesland = meta_author.query("Bundesland == @bundesland")
    schule_authors.loc[gnd_link, 'Schule_'+bundesland] = meta_author_bundesland.shape[0]

schule_authors = schule_authors.fillna(0)
schule_authors.index.name = 'GND'

  0%|          | 0/674 [00:00<?, ?it/s]

In [ ]:
# Jeden Wert durch Gesamtzahl aller Titel in jeweiligem Bundesland teilen
# schule_authors.loc[:, bundeslaender_prefix] = schule_authors.loc[:, bundeslaender_prefix].apply(lambda x: x / x.sum())

In [ ]:
schule_authors['Schule_Summe'] = schule_authors[bundeslaender_prefix].sum(axis=1)

In [ ]:
schule_authors = schule_authors.sort_values(by = 'Schule_Autor')

In [ ]:
schule_authors.query("Schule_Autor == 'Fried'")

,Schule_Autor,Schule_Baden-Württemberg,Schule_Sachsen-Anhalt,Schule_Hessen,Schule_Hamburg,Schule_Niedersachsen,Schule_Rheinland-Pfalz,Schule_Bayern,Schule_Sachsen,Schule_Saarland,Schule_Mecklenburg-Vorpommern,Schule_Bremen,Schule_Brandenburg,Schule_Nordrhein-Westfalen,Schule_Summe
GND,,,,,,,,,,,,,,,
https://d-nb.info/gnd/115688595,Fried,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
https://d-nb.info/gnd/118703145,Fried,1.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,8.0


In [ ]:
this_test_links = [x for x in test_links if x in schule_authors.index]
schule_authors.loc[this_test_links][[
    'Schule_Autor',
    'Schule_Niedersachsen', 'Schule_Bayern',
    'Schule_Summe'
]]

,Schule_Autor,Schule_Niedersachsen,Schule_Bayern,Schule_Summe
GND,,,,
https://d-nb.info/gnd/118505602,Bachmann,2.0,1.0,18.0
https://d-nb.info/gnd/118516906,Büchner,4.0,1.0,28.0
https://d-nb.info/gnd/118519859,Celan,1.0,1.0,5.0
https://d-nb.info/gnd/118527908,Dürrenmatt,5.0,6.0,33.0
https://d-nb.info/gnd/118536109,Frisch,6.0,3.0,28.0
https://d-nb.info/gnd/118585916,Musil,1.0,1.0,9.0


In [ ]:
schule_authors.shape

(674, 15)

In [ ]:
schule_authors.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/02_schule.csv")

### Import and Merge

In [308]:
schule_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/02_schule.csv", index_col = [0])

In [309]:
cols_to_join = schule_authors.columns.difference(data_authors.columns)
data_authors = data_authors.join(schule_authors[cols_to_join], how = 'outer')

In [310]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'Schule_Autor',
    'Schule_Niedersachsen', 'Schule_Bayern',
    'Schule_Summe',
]]

,Schule_Autor,Schule_Niedersachsen,Schule_Bayern,Schule_Summe
GND,,,,
https://d-nb.info/gnd/118505602,Bachmann,2.0,1.0,18.0
https://d-nb.info/gnd/118516906,Büchner,4.0,1.0,28.0
https://d-nb.info/gnd/118519859,Celan,1.0,1.0,5.0
https://d-nb.info/gnd/118523392,NaN,NaN,NaN,NaN
https://d-nb.info/gnd/118527908,Dürrenmatt,5.0,6.0,33.0
https://d-nb.info/gnd/118536109,Frisch,6.0,3.0,28.0
https://d-nb.info/gnd/118585916,Musil,1.0,1.0,9.0


In [311]:
data_authors.shape

(1236, 59)

# Killy

### Scrape

In [ ]:
!pip3 install google_colab_selenium

In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [349]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

In [350]:
# log in via SUB Göttingen
driver.get("https://dbis.ur.de/dbinfo/warpto.php?bib_id=subgo&color=4&titel_id=11553&url=http%3A%2F%2FHAN.SUB.UNI-GOETTINGEN.DE%2Fhan%2Fverfasser-datenbank%2F")

username = driver.find_element(By.ID, "plainuser")
username.send_keys("000721344262")

password = driver.find_element(By.ID, "password")
password.send_keys("karen1na")

password.send_keys(Keys.RETURN)

In [ ]:
# def get_killy_article_content_from_author_name(author):
#     content = ''
#
#     # nur Killy Literaturlexikon
#     driver.get(f"https://www-1degruyter-1com-1gzs214x40407.han.sub.uni-goettingen.de/database/vdbo/search?query=*&startItem=0&pageSize=10&sortBy=relevance&documentVisibility=all&keywordFacets=dbPublication%7EKilly+Literaturlexikon&databases=vdbo&keywordTypesAndValues=dbPerson%7E{author}&matchAnyTerm=false")
#
#     if "Your access to the De Gruyter site has been temporarily blocked" in driver.find_element(By.TAG_NAME, 'body').text:
#         print("Access blocked")
#         return float('NaN')
#
#     if driver.find_elements(By.LINK_TEXT, author):
#         link = driver.find_element(By.LINK_TEXT, author)
#         driver.execute_script("arguments[0].scrollIntoView(true);", link)
#         time.sleep(random.randint(5, 6))
#         link.click()
#         content = driver.find_element(By.ID, "text-container").text
#         return content
#
#     else:
#         for i in range(2, 20):
#             if driver.find_elements(By.LINK_TEXT, str(i)):
#                 link = driver.find_element(By.LINK_TEXT, str(i))
#                 driver.execute_script("arguments[0].scrollIntoView(true);", link)
#                 time.sleep(4)
#                 link.click()
#                 if driver.find_elements(By.LINK_TEXT, author):
#                     link = driver.find_element(By.LINK_TEXT, author)
#                     driver.execute_script("arguments[0].scrollIntoView(true);", link)
#                     time.sleep(4)
#                     link.click()
#                     content = driver.find_element(By.ID, "text-container").text
#                     return content
#
#     return content

In [ ]:
def get_gnd_link_from_vdbo_page (driver):
  gnd_link = float('NaN')

  if driver.find_elements(By.LINK_TEXT, 'GND'):
    gnd_link = driver.find_element(By.LINK_TEXT, 'GND').get_attribute('href')
    if 'http://' in gnd_link:
      gnd_link = re.sub('http://', 'https://', gnd_link)

  return gnd_link

In [ ]:
def get_article_content_from_vdbo_page (driver):
  article_content = driver.find_element(By.ID, "text-container").text
  return article_content

In [ ]:
def get_name_from_vdbo_page (driver, mode = 'simple'):
    author = float('NaN')

    if mode == 'simple':
      author = driver.find_element(By.TAG_NAME, "h1").text

    if mode == 'full':
      article_content = get_article_content_from_vdbo_page(driver)
      searchresults = re.findall("\\nNamen\\n(.*?)\\n", article_content)
      if len(searchresults) > 0:
          author = searchresults[0]

    return author

In [ ]:
def get_source_from_vdbo_page (driver):
    source = float('NaN')

    article_content = get_article_content_from_vdbo_page(driver)

    if "Verfasserlexikon – Die deutsche Literatur des Mittelalters" in article_content:
      source = "Verfasserlexikon MA"
    elif "Verfasserlexikon – Deutscher Humanismus 1480-1520" in article_content:
        source = "Verfasserlexikon Humanismus 1480–1520"
    elif "Verfasserlexikon – Frühe Neuzeit in Deutschland 1520-1620" in article_content:
      source = "Verfasserlexikon FNZ 1520–1620"
    elif "Verfasserlexikon – Frühe Neuzeit in Deutschland 1620-1720" in article_content:
        source = "Verfasserlexikon FNZ 1520–1620"
    elif "Killy Literaturlexikon – Autoren und Werke" in article_content:
      source = "Killy"

    bandnummer = float('NaN')
    bandnummer_a = re.findall("\nBand\n\\d+", article_content)
    bandnummer_b = re.findall("\nBand \\d+", article_content)
    if len(bandnummer_a) > 0:
      bandnummer = bandnummer_a[0].split("\n")[-1]
    if len(bandnummer_b) > 0:
      bandnummer = bandnummer_b[0].split(" ")[-1]

    if pd.notna(source) and pd.notna(bandnummer):
      source = source + ", Bd. " + bandnummer

    return source

In [ ]:
# gnd_authors_to_killy_authors = {
#     'Becher, Johannes Robert' : 'Becher, Johannes R.',
#     'Blankenburg, Christian Friedrich von': 'Blanckenburg, Friedrich von',
#     'Der Stricker': 'Stricker, Der',
#     'Der von Kürenberg': 'Kürenberg, Der von',
#     'Eckhart, Meister': 'Eckhart von Hochheim',
#     'Erasmus, Desiderius': 'Erasmus von Rotterdam, Desiderius',
#     'Grimmelshausen, Hans Jakob Christoffel von' : 'Grimmelshausen, Hans Jacob Christoph von',
#     'Herder, Johann Gottfried von' : 'Herder, Johann Gottfried',
#     "Hofmann von Hofmannswaldau, Christian" : "Hoffmannswaldau, Christian Hoffmann von",
#     "Immermann, Karl Leberecht" : "Immermann, Karl",
#     'Hoffmann, E. T. A.' : 'Hoffmann, E.T.A.',
#     'Knigge, Adolph, Freiherr': 'Knigge, Adolph von',
#     'Jean Paul' : 'Richter, Johann Paul Friedrich',
#     'Karsch, Anna Luise' : 'Karsch, Anna Louisa',
#     "Lavater, Johann Caspar" : "Lavater, Johann Kaspar",
#     "Schleiermacher, Friedrich": "Schleiermacher, Friedrich Daniel Ernst",
#     "Stolberg-Stolberg, Friedrich Leopold zu": "Stolberg Stolberg, Friedrich Leopold zu",
# }

In [360]:
# data_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv", index_col = [0])

killy = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/06_killy.csv", index_col = [0])
# killy = pd.DataFrame()

# links_to_scrape = [x for x in data_authors.sort_values(by='GND_Autor').index if pd.notna(x) and x not in killy.index]
# links_to_scrape = [x for x in killy.index if pd.isna(killy.loc[x, 'Killy_length_Summe'])]
# links_to_scrape = [x for x in data_authors.sort_values(by='GND_Autor').index if pd.notna(x) and x not in killy.index][:250]
# links_to_scrape = [data_authors.query("GND_Autor == @author").index[0] for author in list(gnd_authors_to_killy_authors.keys()) + list(gnd_authors_to_standard_authors.keys())]
# links_to_scrape = data_authors.sample(n=10).index
# links_to_scrape = test_links

In [ ]:
# for i, gnd_link in enumerate(tqdm(links_to_scrape)):
#   author = data_authors.loc[gnd_link, 'GND_Autor']
#   if pd.isna(author):
#     continue
#   elif author in gnd_authors_to_killy_authors.keys():
#     author_search = gnd_authors_to_killy_authors[author]
#   elif author in gnd_authors_to_standard_authors.keys():
#     author_search = gnd_authors_to_standard_authors[author]
#   else:
#     author_search = author
#
#   try:
#     article_content = get_killy_article_content_from_author_name(author_search)
#     if pd.notna(article_content):
#       article_len = 0 if article_content == '' else len(' '.join(article_content.split("\n")).split(" "))
#       author_simple = get_name_from_vdbo_page(driver, mode = 'simple')
#       author_full = get_name_from_vdbo_page(driver, mode = 'full')
#       source = get_source_from_vdbo_page(driver)

#       killy.at[gnd_link, 'GND_Autor'] = author
#       killy.at[gnd_link, 'Killy_simple_Autor'] = author_simple
#       killy.at[gnd_link, 'Killy_full_Autor'] = author_full
#       killy.at[gnd_link, 'Killy_article'] = article
#       killy.at[gnd_link, 'Killy_length_Summe'] = article_len
#       killy.at[gnd_link, 'VDBO_source'] = source
#       print(f"{gnd_link} {author} ({article_len} Wörter Killy) ({source})")
#       if i%20 == 0:
#         time.sleep(random.randint(8, 12))
#
#   except:
#     print(f"{gnd_link} {author} (Fehler)")
#
# killy = killy.sort_values(by = 'GND_Autor')

In [ ]:
for i in tqdm(range(1, 1500)): # max: 7645
    killy_id = "{:0>{}}".format(i, 4)
    if 'Killy_id' in killy.columns.tolist() and 'killy_'+killy_id in killy['Killy_id'].tolist():
      continue

    driver.get(f"https://www-1degruyter-1com-1gzs214jk03c1.han.sub.uni-goettingen.de/database/VDBO/entry/vdbo.killy.{killy_id}/html")

    if "Your access to the De Gruyter site has been temporarily blocked" in driver.find_element(By.TAG_NAME, 'body').text:
        print("Access blocked")
        continue

    if "nicht finden, die Sie aufgerufen haben" in driver.find_element(By.TAG_NAME, 'body').text:
        print(f"Seite nicht gefunden (Killy-Item: {killy_id})")
        continue

    time.sleep(random.randint(5, 7))

    gnd_link = get_gnd_link_from_vdbo_page(driver)
    if pd.isna(gnd_link):
      gnd_link = 'GND_Placeholder_Killy_' + killy_id
    article_content = get_article_content_from_vdbo_page(driver)
    article_len = 0 if article_content == '' else len(' '.join(article_content.split("\n")).split(" "))
    author_simple = get_name_from_vdbo_page(driver, mode = 'simple')
    author_full = get_name_from_vdbo_page(driver, mode = 'full')
    source = get_source_from_vdbo_page(driver)

    killy.at[gnd_link, 'Killy_id'] = 'killy_' + killy_id
    killy.at[gnd_link, 'Killy_simple_Autor'] = author_simple
    killy.at[gnd_link, 'Killy_full_Autor'] = author_full
    killy.at[gnd_link, 'Killy_article'] = article_content
    killy.at[gnd_link, 'Killy_length_Summe'] = article_len
    killy.at[gnd_link, 'VDBO_source'] = source

    print(f"{gnd_link:<34} {author_simple} ({article_len} Wörter Killy) ({source})")

killy = killy.sort_values(by='Killy_id')

  0%|          | 0/1499 [00:00<?, ?it/s]

https://d-nb.info/gnd/116037970    Davidis, Henriette (539 Wörter Killy) (Killy, Bd. 2)
GND_Placeholder_Killy_1101         ‘De Heinrico’ (384 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/120268345    Dean, Martin R. (661 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/119486725    Decius, Nikolaus (396 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/118810863    Dedecius, Karl (482 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/120533405    Dedekind, Constantin Christian (672 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/118671251    Dedekind, Friedrich (864 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/100101844    Degen, Johann Friedrich (822 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/118524313    Degenhardt, Franz Josef (1894 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/118671278    Dehio, Ludwig (391 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/116054522    Dehmel, Paula (288 Wörter Killy) (Killy, Bd. 2)
https://d-nb.info/gnd/118679236 

In [353]:
killy_gnd_links_correction = {
    'https://d-nb.info/gnd/133328562' : 'https://d-nb.info/gnd/118516906', # Büchner, Georg
    'https://d-nb.info/gnd/1096198584' : 'https://d-nb.info/gnd/118523392' # Dahn, Felix
}

killy = killy.rename(index=killy_gnd_links_correction)

In [354]:
killy.index.name = 'GND'

In [355]:
killy.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/06_killy.csv")

### Import and Merge

In [356]:
killy = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/06_killy.csv", index_col = [0])

In [357]:
killy_join = killy.loc[[x for x in killy.index if pd.notna(x) and 'd-nb' in x]].copy()
killy_join = killy_join.drop("Killy_article", axis = 'columns')

In [358]:
cols_to_join = killy_join.columns.difference(data_authors.columns)
data_authors = data_authors.join(killy_join[cols_to_join], how = 'outer')

In [359]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'Killy_simple_Autor',
    'Killy_length_Summe',
]]

,Killy_simple_Autor,Killy_length_Summe
GND,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",2472.0
https://d-nb.info/gnd/118516906,"Büchner, Georg",3260.0
https://d-nb.info/gnd/118519859,"Celan, Paul",2998.0
https://d-nb.info/gnd/118523392,NaN,NaN
https://d-nb.info/gnd/118527908,NaN,NaN
https://d-nb.info/gnd/118536109,NaN,NaN
https://d-nb.info/gnd/118585916,NaN,NaN


# GND

### Scrape

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
def get_response_from_gnd (gnd_url):
    return requests.get(gnd_url)

In [ ]:
# def get_name_from_gnd_response (response):
#     preferred_name_start = response.text.find('gndo:preferredNameEntityForThePerson')
#     preferred_name_slice = response.text[preferred_name_start:preferred_name_start+300]
#
#     forename = ''
#     surname = ''
#
#     forename_start = preferred_name_slice.find("gndo:forename")
#     if forename_start != -1:
#       forename_slice = preferred_name_slice[(forename_start + 15):(forename_start + 100)]
#       forename = re.findall(""".+?(?=\")""", forename_slice)[0]
#
#     nameaddition_start = preferred_name_slice.find("gndo:nameAddition")
#     if nameaddition_start != -1:
#       nameaddition_slice = preferred_name_slice[(nameaddition_start + 19):(nameaddition_start + 100)]
#       nameaddition = re.findall(""".+?(?=\")""", nameaddition_slice)[0]
#       forename = forename + ' ' + nameaddition
#
#     prefix_start = preferred_name_slice.find("gndo:prefix")
#     if prefix_start != -1:
#       prefix_slice = preferred_name_slice[(prefix_start + 13):(prefix_start + 100)]
#       prefix = re.findall(""".+?(?=\")""", prefix_slice)[0]
#       forename = forename + ' ' + prefix
#
#     surname_start = preferred_name_slice.find("gndo:surname")
#     if surname_start != -1:
#       surname_slice = preferred_name_slice[(surname_start + 14):(surname_start + 100)]
#       surname = re.findall(""".+?(?=\")""", surname_slice)[0]
#
#     personalname_start = preferred_name_slice.find("gndo:personalName")
#     if personalname_start != -1:
#       personalname_slice = preferred_name_slice[(personalname_start + 19):(personalname_start + 100)]
#       personalname = re.findall(""".+?(?=\")""", personalname_slice)[0]
#       surname = personalname
#
#     return [forename, surname]

def get_name_from_gnd_response (response):
  name = float('nan')

  soup = BeautifulSoup(response.text, "html.parser")
  td_elements = soup.findAll('td')

  for i, td_element in enumerate(td_elements):
      if td_element.strong != None and td_element.strong.string == 'Person':
          name = td_elements[i+1].string.strip()

  return name

In [ ]:
def get_gender_from_gnd_response (response):
    gender = float('nan')

    gender_pos_start = response.text.find('vocab/gnd/gender#')
    gender_content = response.text[gender_pos_start+17:gender_pos_start+25]

    if 'female' in gender_content:
      gender = 'female'
    elif 'male' in gender_content:
      gender = 'male'
    elif 'notKnown' in gender_content:
      gender = 'notKnown'
    else:
      gender = gender_content

    return gender

# def get_gender_from_gnd_response (response):
#   gender = float('nan')
#
#   soup = BeautifulSoup(response.text, "html.parser")
#   td_elements = soup.findAll('td')
#
#   for i, td_element in enumerate(td_elements):
#       if td_element.strong != None and td_element.strong.string == 'Geschlecht':
#           gender = td_elements[i+1].string.strip()
#
#   return gender

In [ ]:
def get_lifetime_from_gnd_response (response):
    lifetime = [float('nan'), float('nan')]

    birth_pos = response.text.find('gndo:dateOfBirth ')
    if birth_pos != -1:
        birth_content = response.text[birth_pos+18:birth_pos+22]
        if birth_content.isnumeric():
            lifetime[0] = int(birth_content)

    death_pos = response.text.find('gndo:dateOfDeath ')
    if death_pos != -1:
        death_content = response.text[death_pos+18:death_pos+22]
        if death_content.isnumeric():
            lifetime[1] = int(death_content)

    return lifetime

# def get_lifetime_from_gnd_response (response):
#   lifetime = float('NaN')
#
#   soup = BeautifulSoup(response.text, "html.parser")
#   td_elements = soup.findAll('td')
#
#   for i, td_element in enumerate(td_elements):
#       if td_element.strong != None and td_element.strong.string == 'Zeit':
#           lifetime = td_elements[i+1].string.strip()
#           lifetime = [int(x) for x in re.findall(r'\d+', lifetime)]
#           if len(lifetime) >= 2:
#             birth = lifetime[0]
#             for year in lifetime[1:]:
#               if year > lifetime[0]:
#                   death = year
#                   break
#             lifetime = [birth, death]
#
#   return lifetime

In [ ]:
def get_countries_from_gnd_response (response):
    countries = float('nan')

    soup = BeautifulSoup(response.text, "html.parser")
    td_elements = soup.findAll('td')

    for i, td_element in enumerate(td_elements):
        if td_element.strong != None and td_element.strong.string == 'Land':
            countries = td_elements[i+1].string.strip()
            countries = countries.split('; ')

    return countries

In [ ]:
def get_occupations_from_gnd_response (response):
    occupations = float('nan')

    soup = BeautifulSoup(response.text, "html.parser")
    td_elements = soup.findAll('td')

    for i, td_element in enumerate(td_elements):
        if td_element.strong != None and td_element.strong.string == 'Beruf(e)':
            occupation_links = td_elements[i+1].findAll('a')
            occupations = [x.string for x in occupation_links]

            if len(occupations) == 0: # gibt Rubrik Beruf(e), aber dort keine Links, sondern plain text
                occupations = [td_elements[i+1].string.strip()]

    return occupations

In [ ]:
# data_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv", index_col = [0])

gnd = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/00_gnd.csv", index_col = [0])
# gnd = pd.DataFrame()

links_to_scrape = [x for x in data_authors.index if x not in gnd.index and 'd-nb' in x]
links_to_scrape = links_to_scrape[:20]
# links_to_scrape = data_authors.sample(n=50).index
# links_to_scrape = test_links

In [ ]:
for i, gnd_link in enumerate(tqdm(links_to_scrape)):
  if i%20 == 0:
    time.sleep(random.randint(8, 12))

  response_a = get_response_from_gnd(gnd_link)
  response_b = get_response_from_gnd(gnd_link + '/about/lds')

  # try:
  #    forename, surname = get_name_from_gnd_response(response_b)
  #    if surname == '':
  #      print(f"{gnd_link} Fehler (Name)")
  #      author_name = float('NaN')
  #    elif forename != '':
  #      author_name = surname + ', ' + forename
  #    else:
  #      author_name = surname
  #    print(f"{gnd_link} {author_name}")
  #    gnd.at[gnd_link, 'GND_Autor'] = author_name
  #  except:
  #    print(f"{gnd_link} (Fehler: Name)")

  try:
    gnd.at[gnd_link, 'GND_Autor'] = get_name_from_gnd_response(response_a)
    print(f"{gnd_link} {gnd.at[gnd_link, 'GND_Autor']}")
  except:
    print(f"{gnd_link} (Fehler: Autor)")

  try:
    gnd.at[gnd_link, 'GND_Gender'] = get_gender_from_gnd_response(response_b)
  except:
    print(f"{gnd_link} (Fehler: Gender)")

  try:
    gnd.at[gnd_link, 'GND_Geburtsjahr'] = get_lifetime_from_gnd_response(response_b)[0]
    gnd.at[gnd_link, 'GND_Sterbejahr'] = get_lifetime_from_gnd_response(response_b)[1]
  except:
    print(f"{gnd_link} (Fehler: Geburtsjahr/Sterbejahr)")

  try:
    gnd.at[gnd_link, 'GND_Laender'] = ' + '.join(sorted(get_countries_from_gnd_response(response_a)))
  except:
    print(f"{gnd_link} (Fehler: Länder)")

  try:
    gnd.at[gnd_link, 'GND_Berufe'] = ' + '.join(sorted(get_occupations_from_gnd_response(response_a)))
  except:
    print(f"{gnd_link} (Fehler: Berufe)")

  0%|          | 0/1236 [00:00<?, ?it/s]

https://d-nb.info/gnd/11850021X Abraham, a Sancta Clara
https://d-nb.info/gnd/118646680 Achebe, Chinua
https://d-nb.info/gnd/119369125 Achleitner, Friedrich
https://d-nb.info/gnd/118500422 Achternbusch, Herbert
https://d-nb.info/gnd/138053669 Ade, Maren
https://d-nb.info/gnd/118500775 Adorno, Theodor W.
https://d-nb.info/gnd/118500856 Aeschylus
https://d-nb.info/gnd/118647180 Aesopus
https://d-nb.info/gnd/119293439 Agamben, Giorgio
https://d-nb.info/gnd/118501232 Aichinger, Ilse
https://d-nb.info/gnd/118501259 Ajtmatov, Čingiz
https://d-nb.info/gnd/118884972 Alas, Leopoldo
https://d-nb.info/gnd/118501380 Albee, Edward
https://d-nb.info/gnd/119604108 Alciato, Andreas
https://d-nb.info/gnd/119604108 (Fehler: Länder)
https://d-nb.info/gnd/119604108 (Fehler: Berufe)
https://d-nb.info/gnd/1256774294 Alighieri, Dante
https://d-nb.info/gnd/1256774294 (Fehler: Berufe)
https://d-nb.info/gnd/118869159 Allende, Isabel
https://d-nb.info/gnd/123526183 Almond, David
https://d-nb.info/gnd/118502255 A

In [ ]:
gnd.index.name = 'GND'
gnd = gnd.sort_values(by = 'GND_Autor')

gnd['GND_Autor'] = [re.sub('ä', 'ä', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('á', 'á', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('Č', 'Č', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ć', 'ć', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('é', 'é', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ë', 'ë', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('Ō', 'Ō', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('Ö', 'Ö', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ö', 'ö', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ó', 'ó', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ō', 'ō', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('š', 'š', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ü', 'ü', x) if pd.notna(x) else x for x in gnd['GND_Autor']]
gnd['GND_Autor'] = [re.sub('ž', 'z', x) if pd.notna(x) else x for x in gnd['GND_Autor']]

In [ ]:
laender = [str(x).split(" + ") for x in gnd['GND_Laender']]
laender = [item for sublist in laender for item in sublist]
print(pd.Series(laender).value_counts())

laender_deutsch = ['Deutschland', 'Österreich', 'Schweiz']
GND_deutsch = []
for x in gnd['GND_Laender']:
  if pd.isna(x):
    GND_deutsch.append(x)
  elif any(land in str(x) for land in laender_deutsch):
    GND_deutsch.append(True)
  else:
    GND_deutsch.append(False)
gnd['GND_deutsch'] = GND_deutsch

Deutschland (XA-DE)    867
Österreich (XA-AT)     155
Schweiz (XA-CH)        145
USA (XD-US)            121
Frankreich (XA-FR)     117
                      ... 
Finnland (XA-FI)         1
Afghanistan (XB-AF)      1
Amerika (XD)             1
Kolumbien (XD-CO)        1
Kasachstan (XB-KZ)       1
Name: count, Length: 78, dtype: int64


In [ ]:
berufe = [str(x).split(" + ") for x in gnd['GND_Berufe']]
berufe = [item for sublist in berufe for item in sublist]
print(pd.Series(berufe).value_counts())

berufe_schriftsteller = ['Schriftsteller', 'Lyriker', 'Dramatiker', 'Librettist']
GND_Schriftsteller = []
for x in gnd['GND_Berufe']:
  if pd.isna(x):
    GND_Schriftsteller.append(x)
  elif any(beruf in str(x) for beruf in berufe_schriftsteller):
    GND_Schriftsteller.append(True)
  else:
    GND_Schriftsteller.append(False)
gnd['GND_Schriftsteller'] = GND_Schriftsteller

Schriftsteller       1054
Schriftstellerin      290
Dramatiker            213
Lyriker               196
Librettist            168
                     ... 
Internist               1
Agraringenieur          1
Diplomkaufmann          1
Gymnasiallehrerin       1
Sammler                 1
Name: count, Length: 410, dtype: int64


In [ ]:
gnd.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/00_gnd.csv")

### Import and Merge

In [317]:
gnd = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/00_gnd.csv", index_col = [0])

In [318]:
cols_to_join = gnd.columns.difference(data_authors.columns)
data_authors = data_authors.join(gnd[cols_to_join], how = 'left')

In [319]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'GND_Autor', 'GND_Gender', 'GND_Geburtsjahr', 'GND_Sterbejahr',
    'GND_Laender', 'GND_Berufe', 'GND_deutsch'
]]

,GND_Autor,GND_Gender,GND_Geburtsjahr,GND_Sterbejahr,GND_Laender,GND_Berufe,GND_deutsch
GND,,,,,,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",female,1926.0,1973.0,Italien (XA-IT) + Österreich (XA-AT),Librettistin + Musikerin + Schriftstellerin,True
https://d-nb.info/gnd/118516906,"Büchner, Georg",male,1813.0,1837.0,Deutschland (XA-DE) + Frankreich (XA-FR) + Sch...,Arzt + Dramatiker + Dramatiker + Schriftstelle...,True
https://d-nb.info/gnd/118519859,"Celan, Paul",male,1920.0,1970.0,Frankreich (XA-FR) + Jüdischer Kulturkreis (Re...,Lektor + Lyriker + Schriftsteller + Übersetzer,True
https://d-nb.info/gnd/118523392,"Dahn, Felix",male,1834.0,1912.0,Deutschland (XA-DE),Dramatiker + Erzähler + Historiker + Jurist + ...,True
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich",male,1921.0,1990.0,Deutschland (XA-DE) + Schweiz (XA-CH),Dramatiker + Dramaturg + Grafiker + Librettist...,True
https://d-nb.info/gnd/118536109,"Frisch, Max",male,1911.0,1991.0,Deutschland (XA-DE) + Italien (XA-IT) + Schwei...,Architekt + Dramatiker + Drehbuchautor + Journ...,True
https://d-nb.info/gnd/118585916,"Musil, Robert",male,1880.0,1942.0,Deutschland (XA-DE) + Italien (XA-IT) + Schwei...,Herausgeber + Kritiker + Redakteur + Reserveof...,True


In [320]:
# Gibt es Autor:innen mit unterschiedlichen GND-Links, die den gleichen GND-Namen haben?
data_authors['GND_Autor'].value_counts()[data_authors['GND_Autor'].value_counts() > 1]

Series([], Name: count, dtype: int64)

# Wikipedia

### Scrape

In [ ]:
!pip install pywikibot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.6/713.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 15.4 MB/s eta 0:00:00


In [ ]:
# Code (leicht angepasst) nach: https://github.com/temporal-communities/wiki-metrix

# Illmer, V. J., Soethaert, B., Welz, L., Fischer, F., & Jäschke, R. (2024, Februar 21).
# Literatur im Wikiversum – Eine praktische Annäherung über API-Abfragen und Wikipedia-Metriken.
# DHd 2024 Quo Vadis DH (DHd2024), Passau, Deutschland. https://doi.org/10.5281/zenodo.10698426

pywikibot_config = r"""# -*- coding: utf-8  -*-


mylang = 'de'
family = 'wikipedia'
usernames['wikipedia']['de'] = 'test'"""

with open('user-config.py', 'w', encoding="utf-8") as f:
    f.write(pywikibot_config)

import pywikibot
import requests
import datetime
import urllib.parse

def get_page_stats(page: pywikibot.Page):
    """
    Get page stats for a given page.
    """

    # Handle redirects
    # page = handle_redirect(page)

    page_content = page.get(force=True)
    length_in_bytes = len(page_content.encode("utf-8"))
    page_revisions = list(page.revisions(reverse=True))

    data = {
        "title": page.title(),
        "url": page.full_url(),
        "length": length_in_bytes,
        "n_contributors": len(page.contributors()),
        "n_revisions": len(page_revisions),
        "n_extlinks": len(list(page.extlinks())),
        "n_langlinks": len(page.langlinks()),
        "n_links": len(list(page.linkedPages())),
        "n_linkshere": len(
            list(page.linkedPages(namespaces=[0], follow_redirects=False))
        ),  # Article namespace only (0)
        "n_categories": len(list(page.categories())),
        "pageviews_365d": get_pageviews(page, days=365),
        "pageviews_730d": get_pageviews(page, days=730),
        "pageviews_1825d": get_pageviews(page, days=1825),
        "first_revision": page_revisions[0].timestamp,
    }

    MW_API_LIMIT = 500
    # Give warning if any value is at the limit
    for key, value in data.items():
        if value == MW_API_LIMIT:
            print(f"Warning: {key} at limit {MW_API_LIMIT}.")

    return data

# Use Wikimedia Pageviews REST API to get pageviews
def get_pageviews(page: pywikibot.Page, days=365):
    lang = page.site.code
    site = page.site.family.name

    # Wikimedia REST API
    # https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews
    # https://wikimedia.org/api/rest_v1/
    end_date = datetime.date.today() - datetime.timedelta(days=2)  # Two days ago
    start_date = end_date - datetime.timedelta(days=days)  # Two days minus [days] ago

    agent_type = "user"  # user, bot, spider, all-agents
    title_uri = urllib.parse.quote(
        page.title(underscore=True, with_section=False), safe=""
    )  # URI-encoded title, no safe characters
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{lang}.{site}/all-access/{agent_type}/{title_uri}/monthly/{start_date.strftime('%Y%m%d')}/{end_date.strftime('%Y%m%d')}"

    user_agent = f"wiki-metrix (https://github.com/temporal-communities/wiki-metrix) requests/{requests.__version__}"
    response = requests.get(url, headers={"User-Agent": user_agent})

    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} {response.reason}")

    data = response.json()
    pageviews_sum = sum(filter(None, [item["views"] for item in data["items"]]))

    return pageviews_sum

def add_page_stats_to_df (df, wikipedia_article_column = 'Wikipedia_Artikel'):
    site = pywikibot.Site('de', 'wikipedia')  # The site we want to run our bot on
    wikiresults_joined_df = pd.DataFrame()

    for article in tqdm(df[wikipedia_article_column]):
        page = pywikibot.Page(site, article)
        wikiresults = get_page_stats(page)

        wikiresults_df = pd.DataFrame(pd.Series(wikiresults)).T
        wikiresults_joined_df = pd.concat([wikiresults_joined_df, wikiresults_df])

    wikiresults_joined_df = wikiresults_joined_df.reset_index(drop=True)
    df = df.join(wikiresults_joined_df)

    return df

In [ ]:
site = pywikibot.Site('de', 'wikipedia')

In [ ]:
gnd_authors_to_wiki_authors = {
    'Alciato, Andreas': 'Andrea Alciato',
    'Bachtin, Michail Michajlovič' : 'Michail Michailowitsch Bachtin',
    'Blankenburg, Christian Friedrich von': 'Christian Friedrich von Blanckenburg',
    'Cao, Xueqin': 'Cao Xueqin',
    'De Cesco, Federica': 'Federica de Cesco',
    'De Man, Paul': 'Paul de Man',
    'Fiedler, Leslie A.': 'Leslie Fiedler',
    'Gao, Xingjian': 'Gao Xingjian',
    'Gliḳl bas Judah Leib': 'Glikl bas Judah Leib',
    "Gogolʹ, Nikolaj Vasilʹevič": 'Nikolai Wassiljewitsch Gogol',
    'Iolande, von Vianden': 'Yolanda von Vianden',
    'Kittler, Friedrich A.': 'Friedrich Kittler',
    'Lao, She': 'Lao She',
    'Maḥfūẓ, Naǧīb': 'Nagib Mahfuz',
    'Nabokov, Vladimir Vladimirovič': 'Vladimir Nabokov',
    "O'Dell, Scott": 'Scott O’Dell',
    "Orléans, Elisabeth Charlotte d'": "Élisabeth Charlotte d’Orléans",
    "Solženicyn, Aleksandr Isaevič": "Alexander Issajewitsch Solschenizyn",
    "Strugackij, Boris": "Arkadi und Boris Strugazki",
    "Šerstjanoj, Valerij": "Valeri Scherstjanoi",
    "Werner, der Gärtner": "Wernher der Gartenaere",
    "Winter, Léon de": "Leon de Winter",
    "ʿOz, Amos": "Amos Oz"
}

In [ ]:
# data_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv", index_col = [0])

wiki = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/04_wiki.csv", index_col = [0])
# wiki = pd.DataFrame()

links_to_scrape = [x for x in data_authors.query("GND_Autor.notna()").sort_values(by='GND_Autor').index and x not in wiki.index]
# links_to_scrape = [data_authors.query("GND_Autor == @x").index[0] for x in list(gnd_authors_to_standard_authors.keys()) + list(gnd_authors_to_wiki_authors.keys())]
# links_to_scrape = test_links

In [ ]:
for gnd_link in tqdm(links_to_scrape):
  author = data_authors.loc[gnd_link, 'GND_Autor']

  if pd.isna(author):
    continue
  elif author in gnd_authors_to_wiki_authors.keys():
    author_search = gnd_authors_to_wiki_authors[author]
  elif author in gnd_authors_to_standard_authors.keys():
    author_search = gnd_authors_to_standard_authors[author]
  else:
    author_search = author
  author_search = ' '.join(str(author_search).split(', ')[::-1])

  page = pywikibot.Page(site, author_search)
  if page.isRedirectPage():
    page = page.getRedirectTarget()

  try:
    wiki_author = get_page_stats(page)
    wiki_author['GND_Autor'] = author
    wiki_author['Wiki_Status'] = 'page_found'
    wiki_author = pd.DataFrame(pd.Series(wiki_author)).T
    wiki_author.index = [gnd_link]
  except:
    wiki_author = pd.DataFrame(index = [gnd_link])
    wiki_author['GND_Autor'] = author
    wiki_author['Wiki_Status'] = 'page_not_found'

  if wiki_author['Wiki_Status'].tolist()[0] == 'page_not_found':
    print(f"{gnd_link} {author} (Seite nicht gefunden)")
  else:
    print(f"{gnd_link} {author} ({wiki_author['pageviews_1825d'].tolist()[0]} pageviews)")

  wiki_author.columns = [x if 'GND_' in x or 'Wiki_' in x else 'Wiki_'+x for x in wiki_author.columns]

  wiki = wiki.drop(gnd_link, errors='ignore')
  wiki = pd.concat([wiki, wiki_author])

  0%|          | 0/59 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pywikibot/page/_links.py:608: UserWarning: Site wikipedia:be-tarask instantiated using different code "be-x-old"
  link._site = pywikibot.Site(lang, source.family.name)
/usr/local/lib/python3.10/dist-packages/pywikibot/page/_links.py:608: UserWarning: Site wikipedia:no instantiated using different code "nb"
  link._site = pywikibot.Site(lang, source.family.name)


https://d-nb.info/gnd/118647180 Aesopus (377934 pageviews)
https://d-nb.info/gnd/11850374X Apuleius, Madaurensis (85381 pageviews)
https://d-nb.info/gnd/119025698 Ava, Frau (16651 pageviews)
https://d-nb.info/gnd/11867191X Dietmar, von Aist (17029 pageviews)
https://d-nb.info/gnd/118528823 Eckhart, Meister (468256 pageviews)
https://d-nb.info/gnd/118819313 Elisabeth, Nassau-Saarbrücken, Gräfin (12547 pageviews)
https://d-nb.info/gnd/118530666 Erasmus, Desiderius (652964 pageviews)
https://d-nb.info/gnd/118532766 Feuerbach, Paul Johann Anselm, Ritter von (52945 pageviews)
https://d-nb.info/gnd/118534556 Fouqué, Friedrich de La Motte- (32254 pageviews)
https://d-nb.info/gnd/118540947 Gottfried, von Straßburg (48522 pageviews)
https://d-nb.info/gnd/118544918 Hahn-Hahn, Ida, Gräfin (33812 pageviews)
https://d-nb.info/gnd/118546228 Hartmann, von Aue (117401 pageviews)
https://d-nb.info/gnd/118709747 Heinrich, der Gleißner (5895 pageviews)
https://d-nb.info/gnd/118535102 Heinrich, von Meißen

In [ ]:
wiki = wiki.sort_values(by = 'GND_Autor')
wiki.index.name = 'GND'

exceptions = ['Wiki_title', 'Wiki_url', 'Wiki_first_revision', 'Wiki_Status']
fill_columns = [x for x in wiki if x not in exceptions]
wiki[fill_columns] = wiki[fill_columns].fillna(0)

In [ ]:
wiki.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/04_wiki.csv")

### Import and Merge

In [ ]:
wiki = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/04_wiki.csv", index_col = [0])

In [ ]:
cols_to_join = wiki.columns.difference(data_authors.columns)
data_authors = data_authors.join(wiki[cols_to_join], how = 'left')

In [ ]:
data_authors['Wiki_pageviews_Summe'] = data_authors['Wiki_pageviews_1825d']
data_authors['Wiki_length_Summe'] = data_authors['Wiki_length']

In [ ]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'Wiki_title',
    'Wiki_pageviews_Summe', 'Wiki_length_Summe'
]]

,Wiki_title,Wiki_pageviews_Summe,Wiki_length_Summe
GND,,,
https://d-nb.info/gnd/118505602,Ingeborg Bachmann,997388,66902
https://d-nb.info/gnd/118516906,Georg Büchner,900230,42336
https://d-nb.info/gnd/118519859,Paul Celan,552809,53030
https://d-nb.info/gnd/118523392,Felix Dahn,68278,29834
https://d-nb.info/gnd/118527908,Friedrich Dürrenmatt,1221299,56558
https://d-nb.info/gnd/118536109,Max Frisch,899587,113706
https://d-nb.info/gnd/118585916,Robert Musil,365460,67374


# BDSL

### Scrape

In [ ]:
!pip3 install google_colab_selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [ ]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# log in via SUB Göttingen
driver.get("http://han.sub.uni-goettingen.de/han/BDSL")

username = driver.find_element(By.ID, "plainuser")
username.send_keys("000721344262")

password = driver.find_element(By.ID, "password")
password.send_keys("karen1na")

password.send_keys(Keys.RETURN)

In [ ]:
gnd_authors_to_bdsl_authors = {
    'Kittler, Friedrich A.': 'Friedrich Kittler',
}

In [ ]:
# data_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv", index_col = [0])

bdsl = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/03_bdsl.csv", index_col = [0])
# bdsl = pd.DataFrame()

links_to_scrape = [x for x in data_authors.query("GND_Autor.notna()").sort_values(by='GND_Autor').index and x not in bdsl.index]
# links_to_scrape = [data_authors.query("GND_Autor == @x").index[0] for x in list(gnd_authors_to_bdsl_authors.keys()) + list(gnd_authors_to_standard_authors.keys())]
# links_to_scrape = test_links

In [ ]:
for gnd_link in tqdm(links_to_scrape):
  author = data_authors.loc[gnd_link, 'GND_Autor']

  if pd.isna(author):
    continue
  elif author in gnd_authors_to_bdsl_authors.keys():
    author_search = gnd_authors_to_bdsl_authors[author]
  elif author in gnd_authors_to_standard_authors.keys():
    author_search = gnd_authors_to_standard_authors[author]
  else:
    author_search = author

  # go to search
  driver.find_element(By.LINK_TEXT, "Suche").click()

  # change dropdown to 'Behandelte Person' # oder doch lieber 'Freitext'??
  dropdown_menu = driver.find_element(By.NAME, "DD1")
  select = Select(dropdown_menu)
  select.select_by_value("4")

  # enter author name and search
  input_field = driver.find_element(By.NAME, "SF1")
  input_field.send_keys(author_search)
  input_field.send_keys(Keys.RETURN)

  # get number of results
  page_source = driver.page_source
  hits_str = re.findall("\d* Titel gefunden", page_source)

  if len(hits_str) == 0 or hits_str[0].startswith(' Titel'):
    hits_int = 0
  else:
    hits_int = int(hits_str[0].split(" ")[0])

  # add results to bdsl_DataFrame
  print(f"{gnd_link} {author} ({hits_int} Treffer)")
  bdsl.at[gnd_link, 'GND_Autor'] = author
  bdsl.at[gnd_link, 'BDSL_Summe'] = hits_int

bdsl['BDSL_Summe'] = bdsl['BDSL_Summe'].fillna(0)
bdsl.index.name = 'GND'

  0%|          | 0/37 [00:00<?, ?it/s]

https://d-nb.info/gnd/119457679 Kittler, Friedrich A. (124 Treffer)
https://d-nb.info/gnd/118647180 Aesopus (0 Treffer)
https://d-nb.info/gnd/11850374X Apuleius, Madaurensis (0 Treffer)
https://d-nb.info/gnd/119025698 Ava, Frau (0 Treffer)
https://d-nb.info/gnd/11867191X Dietmar, von Aist (17 Treffer)
https://d-nb.info/gnd/118528823 Eckhart, Meister (548 Treffer)
https://d-nb.info/gnd/118819313 Elisabeth, Nassau-Saarbrücken, Gräfin (0 Treffer)
https://d-nb.info/gnd/118530666 Erasmus, Desiderius (0 Treffer)
https://d-nb.info/gnd/118532766 Feuerbach, Paul Johann Anselm, Ritter von (0 Treffer)
https://d-nb.info/gnd/118534556 Fouqué, Friedrich de La Motte- (218 Treffer)
https://d-nb.info/gnd/118540947 Gottfried, von Straßburg (808 Treffer)
https://d-nb.info/gnd/118544918 Hahn-Hahn, Ida, Gräfin (65 Treffer)
https://d-nb.info/gnd/118546228 Hartmann, von Aue (973 Treffer)
https://d-nb.info/gnd/118709747 Heinrich, der Gleißner (66 Treffer)
https://d-nb.info/gnd/118535102 Heinrich, von Meißen (

In [ ]:
bdsl.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/03_bdsl.csv")

### Import and Merge

In [ ]:
bdsl = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/03_bdsl.csv", index_col = [0])

In [ ]:
cols_to_join = bdsl.columns.difference(data_authors.columns)
data_authors = data_authors.join(bdsl[cols_to_join], how = 'left')

In [ ]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'GND_Autor', 'BDSL_Summe'
]]

,GND_Autor,BDSL_Summe
GND,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",2218.0
https://d-nb.info/gnd/118516906,"Büchner, Georg",1775.0
https://d-nb.info/gnd/118519859,"Celan, Paul",2951.0
https://d-nb.info/gnd/118523392,"Dahn, Felix",29.0
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich",1075.0
https://d-nb.info/gnd/118536109,"Frisch, Max",1098.0
https://d-nb.info/gnd/118585916,"Musil, Robert",2622.0


# Reclam

### Scrape

Vgl. https://d-nb.info/010784632

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import re

In [ ]:
# def get_response_from_gnd (gnd_url):
#     return requests.get(gnd_url)

In [ ]:
# def get_reclam_hits_for_author_name (author):
#     driver.get("https://portal.dnb.de/opac.htm")

#     soup = BeautifulSoup(response.text, "html.parser")
#     target_span = soup.find('span', id='searchResultShortListPageInfo')
#     if target_span:
#       reclam_hits = target_span.get_text()
#       reclam_hits = reclam_hits.split(" von ")
#       reclam_hits = int(reclam_hits[1])

#    return reclam_hits

In [ ]:
!pip3 install google_colab_selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [ ]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
driver.get("https://portal.dnb.de/opac.htm")
queryfield = driver.find_element(By.ID, "simpleSearchQueryInputInitial")
queryfield.send_keys("Test")
queryfield.send_keys(Keys.RETURN)

In [ ]:
def get_dnb_printview (searchstring):
    driver.get("https://portal.dnb.de/opac.htm")

    expertensuche = driver.find_element(By.ID, "cqlModeCheckbox")
    expertensuche.click()

    queryfield = driver.find_element(By.ID, "simpleSearchQueryInputWithExistingQuery")
    queryfield.clear()
    queryfield.send_keys(searchstring)

    searchbutton = driver.find_element(By.ID, "simpleSearchButton")
    searchbutton.click()

    if driver.find_elements(By.ID, "linkToPrintPreviewShortList"):
      printview_link = driver.find_element(By.ID, "linkToPrintPreviewShortList")
      printview_link.click()
      return driver.page_source
    else:
      return ''

    return driver.page_source

In [ ]:
def get_hits_from_dnb_printview (dnb_printview):
    reclam_hits = 0

    searchresult = re.findall(" von insgesamt \\d*", dnb_printview)
    if len(searchresult) > 0:
        searchresult = searchresult[0].split(" ")
        reclam_hits = int(searchresult[-1])

    return reclam_hits

In [ ]:
def get_reclam_ubs_from_dnb_printview (dnb_printview):
    reclam_ubs = []

    searchresults_a = re.findall("Reclams Universal-Bibliothek ; Nr. \\d+", dnb_printview)
    searchresults_b = re.findall("Reclams Universal-Bibliothek ; Bd. \\d+", dnb_printview)
    searchresults = searchresults_a + searchresults_b
    if len(searchresults) > 0:
        for searchresult in searchresults:
            searchresult = searchresult.split(" ")
            reclam_ubs.append(int(searchresult[-1]))

    reclam_ubs = sorted(list(set(reclam_ubs)))

    return reclam_ubs

In [ ]:
# data_authors = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv", index_col = [0])

# reclam = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/05_reclam.csv", index_col = [0])
reclam = pd.DataFrame()

links_to_scrape = [x for x in data_authors.query("GND_Autor.notna()").sort_values(by='GND_Autor').index and x not in reclam.index]
# links_to_scrape = data_authors.sample(n=50).index
# links_to_scrape = test_links

In [ ]:
for i, gnd_link in enumerate(tqdm(links_to_scrape)):
  if i > 0 and i%20 == 0:
    time.sleep(random.randint(8, 12))

  author = data_authors.loc[gnd_link, 'GND_Autor']
  if pd.isna(author):
    continue
  else:
    author_search = author

  try:
    reclam.at[gnd_link, 'GND_Autor'] = author

    # searchurl = f"https://portal.dnb.de/opac/simpleSearch?query=partOf%3D010784632+AND+sgt%3DB+AND+atr%3D%22{author_search}%22&cqlMode=true"
    # response = get_response_from_gnd(searchurl)
    # reclam_hits_tokens = get_reclam_hits_from_gnd_response(response)
    # reclam.at[gnd_link, 'Reclam_tokens_Summe'] = reclam_hits_tokens
    # print(f"{gnd_link} {author} ({reclam_hits_tokens} Reclam-Treffer)")

    searchstring = f"partOf=010784632 AND atr={author_search}" # AND sgt=B für nur Belletristik?
    dnb_printview = get_dnb_printview(searchstring)
    reclam_hits_tokens = get_hits_from_dnb_printview(dnb_printview)
    reclam_ubs = get_reclam_ubs_from_dnb_printview(dnb_printview)
    reclam.at[gnd_link, 'Reclam_tokens_Summe'] = reclam_hits_tokens
    reclam.at[gnd_link, 'Reclam_types_Summe'] = len(reclam_ubs)
    reclam.at[gnd_link, 'Reclam_UBs'] = ' + '.join([str(x) for x in reclam_ubs])
    print(f"{gnd_link} {author} ({reclam_hits_tokens} Reclam-Treffer)")

  except:
    print(f"{gnd_link} {author} (Fehler)")

  0%|          | 0/1236 [00:00<?, ?it/s]

https://d-nb.info/gnd/11850021X Abraham, a Sancta Clara (0 Reclam-Treffer)
https://d-nb.info/gnd/118646680 Achebe, Chinua (0 Reclam-Treffer)
https://d-nb.info/gnd/119369125 Achleitner, Friedrich (0 Reclam-Treffer)
https://d-nb.info/gnd/118500422 Achternbusch, Herbert (0 Reclam-Treffer)
https://d-nb.info/gnd/138053669 Ade, Maren (0 Reclam-Treffer)
https://d-nb.info/gnd/118500775 Adorno, Theodor W. (5 Reclam-Treffer)
https://d-nb.info/gnd/118500856 Aeschylus (27 Reclam-Treffer)
https://d-nb.info/gnd/118647180 Aesopus (3 Reclam-Treffer)
https://d-nb.info/gnd/119293439 Agamben, Giorgio (0 Reclam-Treffer)
https://d-nb.info/gnd/118501232 Aichinger, Ilse (2 Reclam-Treffer)
https://d-nb.info/gnd/118501259 Ajtmatov, Čingiz (5 Reclam-Treffer)
https://d-nb.info/gnd/118884972 Alas, Leopoldo (0 Reclam-Treffer)
https://d-nb.info/gnd/118501380 Albee, Edward (3 Reclam-Treffer)
https://d-nb.info/gnd/119604108 Alciato, Andreas (0 Reclam-Treffer)
https://d-nb.info/gnd/1256774294 Alighieri, Dante (15 Recl

In [ ]:
reclam = reclam.sort_values(by = 'GND_Autor')
reclam.index.name = 'GND'

In [ ]:
reclam.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/05_reclam.csv")

### Import and Merge

In [ ]:
reclam = pd.read_csv("/content/drive/MyDrive/2024.Kanonizität/resources/05_reclam.csv", index_col = [0])

In [ ]:
cols_to_join = reclam.columns.difference(data_authors.columns)
data_authors = data_authors.join(reclam[cols_to_join], how = 'left')

In [ ]:
this_test_links = [x for x in test_links if x in data_authors.index]
data_authors.loc[this_test_links][[
    'GND_Autor',
    'Reclam_tokens_Summe', 'Reclam_types_Summe', 'Reclam_UBs'
]]

,GND_Autor,Reclam_tokens_Summe,Reclam_types_Summe,Reclam_UBs
GND,,,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg",8.0,1.0,8008
https://d-nb.info/gnd/118516906,"Büchner, Georg",65.0,14.0,20 + 6060 + 7733 + 7955 + 8210 + 9347 + 9486 +...
https://d-nb.info/gnd/118519859,"Celan, Paul",0.0,0.0,NaN
https://d-nb.info/gnd/118523392,"Dahn, Felix",0.0,0.0,NaN
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich",2.0,1.0,347
https://d-nb.info/gnd/118536109,"Frisch, Max",5.0,2.0,1131 + 8306
https://d-nb.info/gnd/118585916,"Musil, Robert",12.0,8.0,1146 + 8526 + 18789 + 18797 + 18990 + 18991 + ...


# Check and Export

In [ ]:
exceptions = [
    '_Autor',
    '_title', '_url', '_first_revision', '_Status',
    '_Berufe', '_Geburtsjahr', '_Sterbejahr', '_Gender', '_Laender', '_deutsch', '_Schriftsteller',
    '_id', '_article', '_source',
]
fill_columns = [x for x in data_authors if not any(y in x for y in exceptions)]
data_authors[fill_columns] = data_authors[fill_columns].fillna(0)

data_authors.index.name = 'GND'
data_authors = data_authors.sort_values(by = 'GND_Autor')

In [ ]:
data_authors.shape

(1236, 90)

In [ ]:
data_authors.loc[test_links][[
    'GND_Autor', 'Leselisten_Autor', 'Schule_Autor',
    'GND_Gender', 'GND_Laender',
    'Leselisten_Summe', 'Schule_Summe', 'BDSL_Summe', 'Wiki_pageviews_Summe', 'Reclam_tokens_Summe', 'Killy_length_Summe'
]]

,GND_Autor,Leselisten_Autor,Schule_Autor,GND_Gender,GND_Laender,Leselisten_Summe,Schule_Summe,BDSL_Summe,Wiki_pageviews_Summe,Reclam_tokens_Summe,Killy_length_Summe
GND,,,,,,,,,,,
https://d-nb.info/gnd/118505602,"Bachmann, Ingeborg","Bachmann, Ingeborg",Bachmann,female,Italien (XA-IT) + Österreich (XA-AT),78.389635,18.0,2218.0,997388,8.0,2472.0
https://d-nb.info/gnd/118516906,"Büchner, Georg","Büchner, Georg",Büchner,male,Deutschland (XA-DE) + Frankreich (XA-FR) + Sch...,94.503444,28.0,1775.0,900230,65.0,3260.0
https://d-nb.info/gnd/118519859,"Celan, Paul","Celan, Paul",Celan,male,Frankreich (XA-FR) + Jüdischer Kulturkreis (Re...,59.005877,5.0,2951.0,552809,0.0,2998.0
https://d-nb.info/gnd/118523392,"Dahn, Felix","Dahn, Felix",NaN,male,Deutschland (XA-DE),1.000000,0.0,29.0,68278,0.0,953.0
https://d-nb.info/gnd/118527908,"Dürrenmatt, Friedrich","Dürrenmatt, Friedrich",Dürrenmatt,male,Deutschland (XA-DE) + Schweiz (XA-CH),51.812558,33.0,1075.0,1221299,2.0,4299.0
https://d-nb.info/gnd/118536109,"Frisch, Max","Frisch, Max",Frisch,male,Deutschland (XA-DE) + Italien (XA-IT) + Schwei...,63.029624,28.0,1098.0,899587,5.0,3088.0
https://d-nb.info/gnd/118585916,"Musil, Robert","Musil, Robert",Musil,male,Deutschland (XA-DE) + Italien (XA-IT) + Schwei...,61.488971,9.0,2622.0,365460,12.0,3571.0


In [ ]:
data_authors.query("Wiki_pageviews_Summe.isna()")[[
    'GND_Autor', 'Wiki_pageviews_Summe', 'Wiki_length_Summe'
]]

,GND_Autor,Wiki_pageviews_Summe,Wiki_length_Summe
GND,,,


In [ ]:
data_authors['GND_Gender'].value_counts()

GND_Gender
male        909
female      270
notKnown     57
Name: count, dtype: int64

In [ ]:
data_authors.query("GND_Gender == 'notKnown'")['GND_Autor']

GND
https://d-nb.info/gnd/118505025              Auerbach, Erich
https://d-nb.info/gnd/119621290             Bidermann, Jakob
https://d-nb.info/gnd/119237873               Butler, Judith
https://d-nb.info/gnd/123728134               Creech, Sharon
https://d-nb.info/gnd/118887203                 De Man, Paul
https://d-nb.info/gnd/118619284                 Der Stricker
https://d-nb.info/gnd/118567756            Der von Kürenberg
https://d-nb.info/gnd/118922335                  Duden, Anne
https://d-nb.info/gnd/128923865              Durzak, Manfred
https://d-nb.info/gnd/123172047              Einzmann, Nadja
https://d-nb.info/gnd/115646736           Fiedler, Leslie A.
https://d-nb.info/gnd/119220571               Frayn, Michael
https://d-nb.info/gnd/118693549        Friedrich, von Hausen
https://d-nb.info/gnd/119401827             Gahse, Zsuzsanna
https://d-nb.info/gnd/118845497                Gao, Xingjian
http://d-nb.info/gnd/142026263               Geske, Matthias
https://d-nb.info/gn

In [ ]:
data_authors.query("GND_Gender == '<http://'")['GND_Autor']

Series([], Name: GND_Autor, dtype: object)

In [ ]:
data_authors.to_csv("/content/drive/MyDrive/2024.Kanonizität/resources/data_authors.csv")